In [2]:
import h5py
import numpy as np
import random
from typing import *
from tqdm import tqdm
from sda.mcs import KolmogorovFlow
import glob
from utils import *
import torch


In [3]:
n_inits = 10

In [4]:
# def aggregate():
#     files = sorted(PATH.glob('data/x_*.npy'))
#     length = len(files)

#     i = int(0.8 * length)
#     j = int(0.9 * length)

#     splits = {
#         'train': files[:i],
#         'valid': files[i:j],
#         'test': files[j:],
#     }

#     for name, files in splits.items():
#         with h5py.File(PATH / f'data/{name}.h5', mode='w') as f:
#             f.create_dataset(
#                 'x',
#                 shape=(len(files), 64, 2, 64, 64), #time steps, channels, lat, lon
#                 dtype=np.float32,
#             )

#             for i, x in enumerate(map(np.load, files)):
#                 f['x'][i] = KolmogorovFlow.coarsen(torch.from_numpy(x), 4)
scratch = '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/hf/'
files = sorted(glob.glob(scratch + 'uv*.npy'))


In [5]:
files

['/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/hf/uv_b00_t000065.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/hf/uv_b00_t000066.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/hf/uv_b00_t000067.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/hf/uv_b00_t000068.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/hf/uv_b00_t000069.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/hf/uv_b00_t000070.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/hf/uv_b00_t000071.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/hf/uv_b00_t000072.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/hf/uv_b00_t000073.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/hf/uv_b00_t000074.npy',
 '/lustre/fsw/portfolios/nvr/projects/nv

In [6]:
i = int(0.8 * n_inits)
j = int(0.9 * n_inits)

In [ ]:
       'train': files[:i],
        'valid': files[i:j],
        'test': files[j:],

In [7]:
splits = {
    'train': (0, i),
    'valid': (i,j),
    'test': (j,n_inits)
}

In [21]:
for k in range(n_inits):
    files = glob.glob(scratch + f'/uv_b{k:02d}*.npy')


In [22]:
files

['/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/uv_hf/uv_b10_t000076.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/uv_hf/uv_b10_t000069.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/uv_hf/uv_b10_t000083.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/uv_hf/uv_b10_t000096.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/uv_hf/uv_b10_t000093.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/uv_hf/uv_b10_t000080.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/uv_hf/uv_b10_t000120.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/uv_hf/uv_b10_t000125.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/uv_hf/uv_b10_t000084.npy',
 '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/uv_hf/uv_b10_t000111.npy',
 '/lustre/

In [8]:
with h5py.File(scratch+ 'all.h5', mode='w') as f:
            f.create_dataset(
                'x',
                shape=(n_inits* 100 , 64, 2, 256, 256), #time steps, channels, lat, lon
                dtype=np.float32,
            )
            for k in range(n_inits,):
                files = glob.glob(scratch + f'/uv_b{k:02d}*.npy')
                for i, x in enumerate(map(np.load, tqdm(files, ncols=88))):
                    f['x'][k*100:(k+1)*100,i,:,:,:] = x[:,0,:,:,:]

  2%|▊                                                   | 1/64 [00:02<03:02,  2.90s/it]

 45%|███████████████████████                            | 29/64 [01:26<01:04,  1.85s/it]

In [35]:
splits.items()

dict_items([('train', (0, 880)), ('valid', (880, 990)), ('test', (990, 1100))])

In [43]:

with h5py.File(scratch+ 'all.h5', mode='r') as f:
     for name, ind in splits.items():
          with h5py.File(scratch + f'{name}.h5', mode='w') as g:
               g.create_dataset(
                    'x',
                    shape=(ind[1]-ind[0], 64, 2, 256,256),
                    dtype=np.float32)
               g['x'][:] = KolmogorovFlow.coarsen(torch.from_numpy(f['x'][ind[0]:ind[1]]), 4) #KolmogorovFlow.coarsen(torch.from_numpy(f['x'][ind[0]:ind[1]]), 4)

          

In [40]:
splits.items()

dict_items([('train', (0, 880)), ('valid', (880, 990)), ('test', (990, 1100))])

In [42]:
with h5py.File(scratch+ '/train.h5', mode='r') as f:
    print(f['x'][0].shape)
    print(f['x'][0])


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov/data/uv_hf/train.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [13]:
files = sorted(PATH.glob('data/rozx_*.npy'))
length = len(files)

i = int(0.8 * length)
j = int(0.9 * length)

splits = {
    'train': files[:i],
    'valid': files[i:j],
    'test': files[j:],
}


In [15]:

for name, files in splits.items():
    with h5py.File(PATH / f'data/roz{name}.h5', mode='w') as f:
        f.create_dataset(
            'x',
            shape=(len(files), 64, 2, 64, 64), #time steps, channels, lat, lon
            dtype=np.float32,
        )

        for i, x in enumerate(map(np.load, files)):
            f['x'][i] = KolmogorovFlow.coarsen(torch.from_numpy(x), 4)
